# <div style='text-align: center;'> **Projet Gaming** - Partie 1 : Extraction des données via les API *IGDB*, API du *Steam Store*, *Steam Web API* et *SteamSpy* </div>

## Introduction

Ce projet consiste en une série d'analyses appliquées à un jeu de données concernant l'**industrie du jeu vidéo**. 
L'objectif principal est de mettre en pratique des concepts de data science en Python et de démontrer ma capacité à suivre les évolutions technologiques ; il s'agit également pour moi de développer mes compétences dans la gestion de problématiques métier, afin d'élargir ma culture professionnelle tout en produisant des insights pertinents pour l'industrie du jeu vidéo.

Ici, je me suis focalisée sur une catégorie de jeux : **les jeux RGP** (*Role-Playing Games* ou *jeux de rôle* en français), un genre de jeu où les joueurs prennent le contrôle de personnages fictifs dans un monde virtuel.

Afin de travailler ma capacité à manipuler des **données provenant de sources externes**, j'ai décidé de collecter mes données depuis différentes **API** : l'API *IGDB (Twitch)*, l'API du *Steam Store*, *Steam Web API* et enfin *SteamSpy*. Ces interfaces permettent d'extraire des données détaillées sur l'industrie du jeu vidéo relatives à différents aspects du marché, tels que les comportements d'achat, les préférences des joueurs, les performances commerciales des jeux, etc.  
**La première phase de ce projet se concentre sur l'extraction de ces données.**  

Les étapes suivantes se concentreront sur l’analyse de ces données en répondant à une série de questions, qui seront détaillées au fur et à mesure du projet. Parmi ces questions, on retrouve notamment :
- **Analyses descriptives** : par exemple, la corrélation entre les prix et les dates de sortie, les plateformes et les modes de jeu.
- **Analyses prédictives** (approche statistique + approche Machine Learning) : par exemple, évaluation de l'impact de l'année de sortie, de la note, du prix et du nombre de recommandations positives sur la popularité des jeux.
- **Clustering** et **segmentation** sur des données textuelles **(NLP avec scoring)** : par exemple, identification de groupes de jeux similaires en fonction de leurs mécaniques, analysées à partir des résumés.

## Importation des bibliothèques


In [17]:
import requests      # Pour effectuer des requêtes HTTP
import datetime      # Pour manipuler les dates
import pandas as pd  # Pour manipuler les données tabulaires
import os            # Pour gérer les chemins de fichiers
import time          # Pour introduire des délais entre les requêtes
from bs4 import BeautifulSoup  # Pour parser et extraire des données à partir de pages HTML
import json          # Pour lire et écrire des fichiers JSON
from urllib.parse import quote  # Pour encoder des chaînes de texte dans une URL

## Importation des données

### Métadonnées issue de l'API *IGDB (Twitch)*
L’API ***IGDB** (Internet Game Database)*, accessible *via* *Twitch*, est une base de données extrêmement complète sur les jeux vidéo. Elle permet en particulier d’extraire un ensemble de métadonnées (données descriptives) très utiles sur les jeux.  

Pour y accéder, il est nécessaire d'avoir un compte *Twitch Developer* pour obtenir un client_id et un access_token. Les requêtes se font en POST – et non en GET, pourtant plus courant pour la lecture de données dans les API de type REST telles que l'API *IGDB*. Ce choix s’explique par la syntaxe spécifique de l’API *IGDB* (nommée *Apicalypse*, proche du SQL), qui nécessite souvent des requêtes complexes. Le corps de la requête POST permet de structurer cette complexité, ce qui ne serait pas possible avec une requête GET classique.

#### Authentification à l'API

In [18]:
CLIENT_ID = "hkmuvrxev3jgb6077y6v5b719ot319"
ACCESS_TOKEN = "fq9tkosl71ghep16hnjdfietime65g"

# En-têtes HTTP pour l'authentification auprès de l'API
headers = {
    "Client-ID": CLIENT_ID,
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

#### Requête pour l'extraction des données de l'API

Ici, j'extrais une série de variables **pour les 75 jeux RPG les mieux notés** : 

- Le nom du jeu vidéo ;
- Le résumé ou la description du jeu ;
- La date de première sortie du jeu ;
- Les noms des plateformes sur lesquelles le jeu est disponible ;
- La note agrégée du jeu (moyenne des critiques professionnelles) ;
- Les noms des entreprises impliquées dans le développement/édition du jeu ;
- Les différents modes de jeu disponibles (solo, multijoueur, etc.).

In [19]:
url = "https://api.igdb.com/v4/games" # URL de l'endpoint des jeux de l'API IGDB

# Corps de la requête en syntaxe Apicalypse
# N.B. : Apicalypse est un langage très simplifié de requête développé par les créateurs d’IGDB pour interroger facilement leur base de données
body = """
fields name, summary, first_release_date, platforms.name, aggregated_rating,
involved_companies.company.name, game_modes.name;
where genres = 12 & aggregated_rating != null;
sort aggregated_rating desc;
limit 75;
"""

# Envoi de la requête POST à l'API
# N.B. : POST permet d'envoyer des corps de requête plus longues et plus complexes que GET ; IGDB a conçu son API pour utiliser POST comme méthode standard de requêtage
response = requests.post(url, headers=headers, data=body)

# Liste qui contiendra les données de jeux formatées
games_data = []

# Vérification que la requête a réussi (code 200)
if response.status_code == 200:
    # Conversion de la réponse JSON en liste de dictionnaires Python
    data = response.json()
    
    for game in data:
        # Dictionnaire pour stocker les informations du jeu actuel
        game_info = {}
        
        game_info["Nom"] = game['name']
        game_info["Résumé"] = game.get('summary', 'Non disponible')
        
        # Traitement de la date de sortie
        release_date = game.get('first_release_date')
        if release_date:
            # Conversion du timestamp Unix en format de date lisible
            release_date = datetime.datetime.fromtimestamp(release_date, datetime.UTC).strftime('%Y-%m-%d')
        else:
            release_date = "Non disponible"
        game_info["Date de sortie"] = release_date
        
        game_info["Note"] = round(game.get('aggregated_rating'), 2)
        
        # Extraction et formatage des plateformes disponibles
        platforms = [p['name'] for p in game.get('platforms', [])]
        game_info["Plateformes"] = ', '.join(platforms) if platforms else 'Non disponible'
        
        # Extraction et formatage des entreprises impliquées (développeurs/éditeurs)
        companies = [c['company']['name'] for c in game.get('involved_companies', [])]
        game_info["Développeurs/Éditeurs"] = ', '.join(companies) if companies else 'Non disponible'
        
        # Extraction et formatage des modes de jeu
        game_modes = [m['name'] for m in game.get('game_modes', [])]
        game_info["Modes de jeu"] = ', '.join(game_modes) if game_modes else 'Non disponible'
        
        # Ajout des informations du jeu à la liste principale
        games_data.append(game_info)

    
    # Création d'un dataframe à partir de la liste de dictionnaires
    df_IGDB = pd.DataFrame(games_data)
    
    # Utilisation du répertoire courant pour enregistrer le fichier
    csv_path = "IGDB_jeux_rpg.csv"
    
    # Exportation du dataframe au format CSV
    df_IGDB.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"Les données ont été enregistrées dans '{os.path.abspath(csv_path)}'")
    
    display(df_IGDB.head())  
else:
    print(f"Erreur {response.status_code} : {response.text}")

Les données ont été enregistrées dans 'C:\Users\leaam\IGDB_jeux_rpg.csv'


,Nom,Résumé,Date de sortie,Note,Plateformes,Développeurs/Éditeurs,Modes de jeu
0,EverQuest: Evolution,contains the original EverQuest game with all ...,2003-08-25,100.00,PC (Microsoft Windows),Non disponible,Massively Multiplayer Online (MMO)
1,Dark Age of Camelot,Dark Age of Camelot is a 3D medieval fantasy M...,2001-10-10,100.00,PC (Microsoft Windows),Mythic Entertainment,Massively Multiplayer Online (MMO)
2,Dragon Age: Origins Collector's Edition,Dragon Age: Origins Collector's Edition contai...,2009-12-03,98.00,PlayStation 3,"Electronic Arts, BioWare",Single player
3,World of Warcraft: Wrath of the Lich King,"World of Warcraft: Wrath of the Lich King, oft...",2008-11-13,96.67,"PC (Microsoft Windows), Mac",Blizzard Entertainment,Massively Multiplayer Online (MMO)
4,Baldur's Gate: Siege of Dragonspear - Collecto...,The Collector's Edition features a limited-edi...,2016-03-31,95.00,"PC (Microsoft Windows), Mac","Overhaul Games, Beamdog",Single player


In [20]:
# display(list(df_IGDB["Nom"]))

L'API *IGDB* fournit les **métadonnées** des jeux mais ne donne pas de **statistiques relatives au marché**.  
Les métadonnées peuvent être mixées avec les données statistiques d'autres API pour fournir des détails commerciaux, des informations relatives au comportement des joueurs, etc.  
Dans cette étude, nous bornons les analyses à une partie seulement des jeux RPG extraits de l'API *IGDB*, **ceux disponibles sur *Steam***. *Steam* est l'une des plus importantes plateformes pour acheter, télécharger et jouer à des jeux vidéo en ligne développée par *Valve Corporation*, avec des millions d'utilisateurs actifs. Les données étudiées ne concernent donc pas les jeux disponibles sur console ou mobile.

### Base de données statistiques issue des API du *Steam Store*, *Steam Web API* (API officielle) et *SteamSpy*

Parmi les 75 jeux ci-dessus, seule une partie est disponible sur *Steam*.

#### Vérification des jeux présents sur *Steam* via l'API de recherche *Steam*
**Avant** d'extraire les données statistiques, un détour par l'**API de recherche de *Steam*** permet d'identifier les jeux présents dans le store.

*Attention :* Certains jeux disponibles sur *Steam* ont une partie de leur base de joueurs en dehors de la plateforme (launcher propriétaire). Pour ces jeux-là, les chiffres *Steam* ne représentent donc qu'une fraction de la population totale, mais nous nous concentrerons ici sur l'activité de cette plateforme.

In [21]:
games = list(df_IGDB["Nom"])

# Liste pour stocker les résultats
games_on_steam = []
games_not_on_steam = []

# Fonction pour normaliser le nom de jeu pour améliorer les chances de trouver une correspondance avec les données de l'API
def normalize_game_name(name):
    name = name.lower()
    
    # Suppression des mots de base pour DLCs et éditions spéciales
    replacements = [
        "collector's edition", "definitive edition", "hd remaster",
        "- blood and wine", ": echoes of", "treasures of the sun",
        "hd-2d remake", "dx", "vr"
    ]
    
    for r in replacements:
        name = name.replace(r, "").strip()
    
    # Suppression des marqueurs d'édition/version avec des caractères spéciaux
    if ":" in name:
        # Attention : Je traite différemment certaines séries de jeux pour lesquelles garder le nom complet est mieux, la partie après le deux-points étant significative pour l'identification
        if any(x in name.lower() for x in ["final fantasy", "dragon quest", "elder scrolls"]):
            pass
        else:
            name = name.split(":")[0].strip()
    
    return name.strip()

# Fonction pour vérifier si un jeu existe sur le Steam Store via l'API de recherche Steam (API "publique" utilisée par le site web de Steam)
def check_steam_store_api(game_name):
    normalized_name = normalize_game_name(game_name)
    encoded_name = quote(normalized_name)
    
    url = f"https://store.steampowered.com/api/storesearch/?term={encoded_name}&l=french&cc=FR"
    
    try:
        response = requests.get(url, headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        
        if response.status_code == 200:
            data = response.json()
            
            if data.get('total', 0) > 0 and 'items' in data:
                for item in data['items']:
                    steam_name = item.get('name', '').lower()
                    app_id = item.get('id')
                    
                    # Vérification d'une correspondance approximative avec le nom normalisé
                    name_parts = normalized_name.split() # Découpe le nom normalisé en mots
                    
                    # Vérification que la majorité des mots du nom original sont dans le résultat Steam
                    word_match_count = sum(1 for word in name_parts if word in steam_name and len(word) > 3) # Décompte du nombre de mots (>3 caractères) présents dans le nom Steam
                    word_match_ratio = word_match_count / len(name_parts) if name_parts else 0 # Calcul d'un ratio de correspondance
                    
                    # Si la correspondance est suffisante (i.e. si nom normalisé entièrement dans le nom Steam ou plus de 50% des mots correspondent ou début du nom correspondant pour les noms plus longs)
                    if (normalized_name in steam_name or
                        word_match_ratio > 0.5 or
                        (len(normalized_name) > 5 and 
                         steam_name.startswith(normalized_name[:5]) and 
                         len(normalized_name) > 8)):
                        return True, app_id, item.get('name')
                
            return False, None, None
        else:
            print(f"Erreur HTTP {response.status_code} pour {game_name}")
            return False, None, None
            
    except Exception as e:
        print(f"Exception lors de la recherche de {game_name}: {str(e)}")
        return False, None, None

# Boucle principale pour vérifier la présence de chaque jeu sur Steam
for index, game in enumerate(games):
    # print(f"[{index+1}/{len(games)}] Vérification de {game}")
    
    is_on_steam, app_id, steam_name = check_steam_store_api(game)
    
    if is_on_steam:
        games_on_steam.append({
            'original_name': game,
            'steam_name': steam_name,
            'app_id': app_id
        })
    else:
        games_not_on_steam.append(game)
    
    time.sleep(0.5) # Pause pour éviter de surcharger l'API


print("\n--- RÉSUMÉ ---")
print(f"Total des jeux vérifiés: {len(games)}")
print(f"Jeux trouvés sur Steam: {len(games_on_steam)}")
print(f"Jeux non trouvés sur Steam: {len(games_not_on_steam)}")

print("\nJeux sur Steam:")
for game in games_on_steam:
    print(f"- {game['original_name']} → {game['steam_name']} (ID: {game['app_id']})")

# Récupération de la liste des jeux sur Steam avec leurs AppID
steam_games = {game['original_name']: game['app_id'] for game in games_on_steam}

# Vérification manuelle : J'ôte le jeu Lunar: Silver Star Harmony, non présent sur Steam et mal associé ici (rappel : J'ai ici choisi des critères maximisant les possibilités de trouver une correspondance mais ils sont peu restrictifs et peuvent induire des coquilles) 
steam_games = {game: app_id for game, app_id in steam_games.items() if game != "Lunar: Silver Star Harmony"}

# Sauvegarde de la liste dans un fichier JSON
with open('steam_jeux_liste.json', 'w', encoding='utf-8') as f:
    json.dump(steam_games, f, ensure_ascii=False, indent=4)

print("Liste sauvegardée dans 'steam_jeux_liste.json'")


--- RÉSUMÉ ---
Total des jeux vérifiés: 75
Jeux trouvés sur Steam: 38
Jeux non trouvés sur Steam: 37

Jeux sur Steam:
- EverQuest: Evolution → EverQuest II (ID: 201230)
- Dragon Age: Origins Collector's Edition → Dragon Age™: The Veilguard (ID: 1845910)
- Baldur's Gate: Siege of Dragonspear - Collector's Edition → Baldur's Gate 3 (ID: 1086940)
- Suikoden → Suikoden I&II HD Remaster Gate Rune and Dunan Unification Wars (ID: 1932640)
- Final Fantasy XVI → FINAL FANTASY XVI (ID: 2515020)
- Dragon's Dogma II → Dragon's Dogma 2 (ID: 2054970)
- Dragon Age: Origins → Dragon Age™: The Veilguard (ID: 1845910)
- Shin Megami Tensei: Persona → Shin Megami Tensei V: Vengeance (ID: 1875830)
- Shin Megami Tensei: Digital Devil Saga 2 → Shin Megami Tensei V: Vengeance (ID: 1875830)
- Atelier Meruru: The Apprentice of Arland → Atelier Meruru ~The Apprentice of Arland~ DX (ID: 936190)
- Baldur's Gate II: Shadows of Amn → Baldur's Gate 3 (ID: 1086940)
- Albion Online → Albion Online (ID: 761890)
- The E

#### Extraction des données statistiques via 3 API différentes
Afin d'accéder à une diversité de **données statistiques** relatives aux jeux étudiés, je fais appel à 3 API : 
- L'**API du *Steam Store*** nous permet d'obtenir les détails commerciaux et descriptifs des jeux. J'extrais ici :
    - Le prix actuel ;
    - Le score Metacritic (note des critiques, de 0 - *critiques généralement défavorables* - à 100 - *critiques généralement favorables*)  
Les scores Metacritic sont très influents dans l'industrie du jeu vidéo et peuvent affecter les ventes. Ils sont souvent utilisés comme un indicateur rapide de la qualité globale d'un jeu selon la presse spécialisée, bien que ces scores ne reflètent pas nécessairement l'opinion des joueurs ou votre expérience personnelle avec le jeu.  
    - Le nombre de recommandations positives.  

- Je passe ensuite par la ***Steam Web API*** (officielle) pour déterminer le nombre de joueurs actuels. Cette API est actuellement le seul endpoint public connu qui donne en temps réel le nombre de joueurs connectés à un jeu donné.  

**N.B.** : L'API ne compte que les joueurs connectés à ce moment précis. Les données sont un screen shot d'un instant i, elles varient selon le moment où le script est lancé (*pour information, les dataframes constitués ici sont réalisés à partir d'un screen shot effectué le samedi 13/04/2025 aux alentours de 18h*).  

- Enfin, j'ai recours à l'**API *SteamSpy*** pour récupérer : 
    - Le nombre de propriétaires du jeu ;
    - Le temps de jeu moyen ;
    - Une approximation des revenus générés par chaque jeu, en faisant le produit du prix du jeu par le nombre médian de propriétaires du jeu.  

*Attention :* L'API *SteamSpy* fournit des **estimations** relatives au comportement des joueurs pour les jeux disponibles sur *Steam*, et non des chiffres officiels. Elle fonctionne en *scrapant* et en échantillonnant les profils publics *Steam*. Bien que non officielles, cette API donne accès à des informations impossibles à obtenir autrement, d'une grande richesse, rendant son utilisation très utile pour des analyses de tendances, de marché, ou de la data viz.

In [23]:
# Chargement de la liste des jeux depuis le fichier JSON
with open('steam_jeux_liste.json', 'r', encoding='utf-8') as f:
    jeux_steam = json.load(f)
    
# Transformation du dictionnaire en liste de dictionnaires pour correspondre au précédent format
jeux_avec_appid = [
    {"nom": nom, "appid": str(appid)} for nom, appid in jeux_steam.items()
]

# URLs des APIs
steam_store_url = "https://store.steampowered.com/api/appdetails" # API du Steam Store
steam_stats_url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/" # Steam Web API (officielle)
steamspy_url = "https://steamspy.com/api.php"  # API SteamSpy

# Configuration de headers pour imiter un navigateur (pour éviter les blocages) 
# N.B. : Ces en-têtes, facultatives, sont utilisées pour contourner les restrictions anti-scraping (beaucoup de sites web, dont Steam, ont des mesures pour détecter et bloquer les scripts automatisés qui collectent des données ; ces scripts sont souvent détectés en observant les en-têtes HTTP des requêtes).
# En configurant ces en-têtes, notre script se fait passer pour un vrai navigateur web (ici Chrome) utilisé par un humain.
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', # Indique quel navigateur et système d'exploitation on "utilise"
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7', # Précise les préférences linguistiques, utile pour avoir des réponses localisées
    'Accept': 'application/json, text/plain, */*', # Définit les types de contenu que le client (i.e. notre script Python) peut traiter (JSON, texte, etc.).
    'Origin': 'https://store.steampowered.com', # Indique d'où provient la requête (certaines API vérifient si les requêtes proviennent de sources légitimes)
    'Referer': 'https://store.steampowered.com/' # Idem
}


# Liste pour stocker les résultats
resultats = []
for jeu in jeux_avec_appid:
    try:
        print(f"Récupération des données pour {jeu['nom']} (AppID: {jeu['appid']})...") 
        
        # Paramètres pour la requête du Store
        store_params = {
            "appids": jeu['appid'],
            "cc": "fr",  # Code pays pour les prix en euros
            "l": "french"  # Langue en français
        }
        
        store_response = requests.get(steam_store_url, params=store_params)
        time.sleep(1.5)  # Délai pour éviter les limitations
        
        # Paramètres pour les statistiques de joueurs actuels via l'API officielle Steam
        stats_params = {
            "appid": jeu['appid']
        }
        
        stats_response = requests.get(steam_stats_url, params=stats_params)
        time.sleep(1.5)  # Délai pour éviter les limitations
        
        # Paramètres pour SteamSpy
        steamspy_params = {
            "request": "appdetails",
            "appid": jeu['appid']
        }
        
        steamspy_response = requests.get(steamspy_url, params=steamspy_params)
        time.sleep(1.5)  # Délai pour éviter les limitations
        
        # Initialisation des variables
        prix = "Non disponible"
        metacritic = "N/A"
        recommandations_positives = "N/A"
        
        # Variables pour SteamSpy
        proprietaires = "N/A"
        temps_jeu_moyen = "N/A"
        revenus_approx = "N/A"
        
        # Traitement des données du Store
        if store_response.status_code == 200:
            store_data = store_response.json()
            if store_data and jeu['appid'] in store_data and store_data[jeu['appid']].get('success', False):
                data = store_data[jeu['appid']]['data']
                
                # Prix
                if 'price_overview' in data:
                    prix = f"{data['price_overview'].get('final')/100:.2f} €"
                elif data.get('is_free', False):
                    prix = "Gratuit"
                
                # Score Metacritic
                if 'metacritic' in data:
                    metacritic = data['metacritic'].get('score', "N/A")
                
                # Recommandations positives
                if 'recommendations' in data:
                    recommandations_positives = data['recommendations'].get('total', "N/A")
        
        # Traitement des statistiques de joueurs actuels (API officielle Steam)
        joueurs_actuels = 0
        if stats_response.status_code == 200:
            stats_data = stats_response.json()
            if 'response' in stats_data and 'player_count' in stats_data['response']:
                joueurs_actuels = stats_data['response']['player_count']
        
        # Traitement des données SteamSpy
        if steamspy_response.status_code == 200:
            steamspy_data = steamspy_response.json()
            
            # Nombre de propriétaires (format: "min-max")
            if 'owners' in steamspy_data:
                proprietaires = steamspy_data.get('owners', "N/A")
            
            # Temps de jeu moyen (en minutes, à convertir en heures)
            if 'average_forever' in steamspy_data:
                temps_minutes = steamspy_data.get('average_forever', 0)
                temps_jeu_moyen = f"{temps_minutes/60:.2f} heures"
            
            # Revenus approximatifs (prix * nombre médian de propriétaires)
            if prix != "Non disponible" and prix != "Gratuit" and proprietaires != "N/A":
                try:
                    # Extraction du prix en nombre
                    prix_numerique = float(prix.replace(' €', ''))
                    
                    # Extraction de la médiane des propriétaires
                    if ' .. ' in proprietaires:
                        min_owners, max_owners = map(int, proprietaires.replace(',', '').split(' .. '))
                        median_owners = (min_owners + max_owners) / 2
                        revenus_approx = f"{prix_numerique * median_owners:,.2f} €"
                except:
                    revenus_approx = "Calcul impossible"
        
        # Ajout au résultat
        resultats.append({
            "Nom": jeu['nom'],
            "AppID": jeu['appid'],
            "Prix": prix,
            "Joueurs Actuels": joueurs_actuels,
            "Score Metacritic": metacritic,
            "Recommandations Positives": recommandations_positives,
            "Propriétaires": proprietaires,
            "Temps de Jeu Moyen": temps_jeu_moyen,
            "Revenus Approximatifs": revenus_approx
        })
        
    except Exception as e:
        print(f"Erreur pour {jeu['nom']}: {str(e)}")
        resultats.append({
            "Nom": jeu['nom'],
            "AppID": jeu['appid'],
            "Prix": "Erreur",
            "Joueurs Actuels": 0,
            "Score Metacritic": "Erreur",
            "Recommandations Positives": "Erreur",
            "Propriétaires": "Erreur",
            "Temps de Jeu Moyen": "Erreur",
            "Revenus Approximatifs": "Erreur"
        })
        
# Création d'un dataframe avec les résultats
df_jeux_steam = pd.DataFrame(resultats)

# Tri par nombre de joueurs actuels (décroissant)
df_jeux_steam = df_jeux_steam.sort_values(by="Joueurs Actuels", ascending=False)

# Sauvegarde et affichage des résultats
print(f"\nDonnées récupérées pour {len(df_jeux_steam)} jeux")

chemin_sauvegarde = 'Steam_jeux_rpg.csv'
df_jeux_steam.to_csv(chemin_sauvegarde, index=False, encoding='utf-8-sig')
print(f"Données sauvegardées dans {chemin_sauvegarde}")

display(df_jeux_steam)

Récupération des données pour EverQuest: Evolution (AppID: 201230)...
Récupération des données pour Dragon Age: Origins Collector's Edition (AppID: 1845910)...
Récupération des données pour Baldur's Gate: Siege of Dragonspear - Collector's Edition (AppID: 1086940)...
Récupération des données pour Suikoden (AppID: 1932640)...
Récupération des données pour Final Fantasy XVI (AppID: 2515020)...
Récupération des données pour Dragon's Dogma II (AppID: 2054970)...
Récupération des données pour Dragon Age: Origins (AppID: 1845910)...
Récupération des données pour Shin Megami Tensei: Persona (AppID: 1875830)...
Récupération des données pour Shin Megami Tensei: Digital Devil Saga 2 (AppID: 1875830)...
Récupération des données pour Atelier Meruru: The Apprentice of Arland (AppID: 936190)...
Récupération des données pour Baldur's Gate II: Shadows of Amn (AppID: 1086940)...
Récupération des données pour Albion Online (AppID: 761890)...
Récupération des données pour The Elder Scrolls: Arena (AppID:

,Nom,AppID,Prix,Joueurs Actuels,Score Metacritic,Recommandations Positives,Propriétaires,Temps de Jeu Moyen,Revenus Approximatifs
10,Baldur's Gate II: Shadows of Amn,1086940,47.99 €,80926,96,655397,"20,000,000 .. 50,000,000",121.70 heures,"1,679,650,000.00 €"
2,Baldur's Gate: Siege of Dragonspear - Collecto...,1086940,47.99 €,80926,96,655380,"20,000,000 .. 50,000,000",121.70 heures,"1,679,650,000.00 €"
20,Final Fantasy XIV Online,39210,9.99 €,28527,83,72881,"2,000,000 .. 5,000,000",517.92 heures,"34,965,000.00 €"
19,Fallout 4 VR,377160,19.99 €,21120,84,259877,"10,000,000 .. 20,000,000",98.37 heures,"299,850,000.00 €"
11,Albion Online,761890,Gratuit,13121,N/A,1869,"2,000,000 .. 5,000,000",67.07 heures,N/A
26,Star Wars: The Old Republic,1286830,Gratuit,5932,85,410,"5,000,000 .. 10,000,000",15.60 heures,N/A
22,Guild Wars 2: Heart of Thorns,1284210,Gratuit,5416,90,338,"2,000,000 .. 5,000,000",37.93 heures,N/A
28,Guild Wars 2: Path of Fire,1284210,Gratuit,5416,90,338,"2,000,000 .. 5,000,000",37.93 heures,N/A
21,Grim Dawn: Definitive Edition,219990,24.99 €,3202,83,83283,"10,000,000 .. 20,000,000",55.65 heures,"374,850,000.00 €"
6,Dragon Age: Origins,1845910,59.99 €,2334,N/A,36556,"2,000,000 .. 5,000,000",50.58 heures,"209,965,000.00 €"


**Remarques** :  
- *Importance des N/A pour le score Metacritic :*  
Metacritic est un site indépendant qui agrège les critiques de jeux vidéo, mais il n’est pas nécessairement associé à tous les jeux sur *Steam*. Certains jeux, notamment ceux indépendants ou plus anciens, n'ont souvent pas été évalués par suffisamment de critiques pour recevoir un score officiel. Metacritic tend à couvrir principalement les titres bénéficiant d'une visibilité médiatique significative, ce qui explique l'absence de scores pour certains jeux.

- *Nombre de joueurs actuels parfois étonnament bas :*   
Des nombres de joueurs actuels quasi nuls voire nuls s'expliquent par l'ancienneté de certains jeux.  
*P.ex.* : Le jeu *Dungeon Siege III: Treasures of the Sun* est paru en 2011. Une vérification manuelle sur *Steam Charts*, site qui recense une série de statistiques sur les jeux disponibles sur *Steam*, permet de confirmer la valeur de 0 joueurs actuels.  

&#8594; **Une vérification manuelle sur *Steam Charts* a été opérée pour vérifier la pertinence de l'output pour tous les jeux pour lesquels le nombre de joueurs actuels trouvé est inférieur à 30.**  
Il n'y a que pour l'*Atelier Sophie 2: The Alchemist of the Mysterious Dream*, sorti en 2022, que cette valeur est plus surprenante. Ceci peut être dû à une erreur de collecte de données ou à un problème lors de l'extraction des statistiques via l'API (il est peu probable en revanche que le problème soit lié à la période de collecte des données peu propice (heure creuse, jour peu actif), le script ayant été relancé à différentes dates avec toujours la même problématique). J'ai fait le choix d'ôter le jeu du dataframe (voir code ci-après).

In [24]:
# Elimination du jeu Atelier Sophie 2: The Alchemist of the Mysterious Dream
df_jeux_steam = df_jeux_steam[df_jeux_steam['Nom'] != 'Atelier Sophie 2: The Alchemist of the Mysterious Dream']

# Nouvelle sauvegarde des résultats
chemin_sauvegarde = 'steam_jeux_rpg.csv'
df_jeux_steam.to_csv(chemin_sauvegarde, index=False, encoding='utf-8-sig')

### Fusion des jeux de données

In [25]:
# Jointure externe droite sur les enregistrements de df_jeux_steam
df_merged = df_IGDB.merge(df_jeux_steam, on='Nom', how='right')
display(df_merged)

# Sauvegarde du dataframe global
csv_path = "jeux_rpg.csv"
df_merged.to_csv(csv_path, index=False, encoding='utf-8-sig')

display(df_merged.head())

,Nom,Résumé,Date de sortie,Note,Plateformes,Développeurs/Éditeurs,Modes de jeu,AppID,Prix,Joueurs Actuels,Score Metacritic,Recommandations Positives,Propriétaires,Temps de Jeu Moyen,Revenus Approximatifs
0,Baldur's Gate II: Shadows of Amn,Every World has conflict. Good and evil. Frien...,2000-09-21,90.00,"Linux, PC (Microsoft Windows), Mac","BioWare, Black Isle Studios, BioWare Edmonton,...","Single player, Co-operative",1086940,47.99 €,80926,96,655397,"20,000,000 .. 50,000,000",121.70 heures,"1,679,650,000.00 €"
1,Baldur's Gate: Siege of Dragonspear - Collecto...,The Collector's Edition features a limited-edi...,2016-03-31,95.00,"PC (Microsoft Windows), Mac","Overhaul Games, Beamdog",Single player,1086940,47.99 €,80926,96,655380,"20,000,000 .. 50,000,000",121.70 heures,"1,679,650,000.00 €"
2,Final Fantasy XIV Online,Take part in an epic and ever-changing Final F...,2013-08-27,85.33,"Xbox Series X|S, PlayStation 3, PlayStation 4,...","Square Enix Product Development Division 3, Sq...","Single player, Multiplayer, Co-operative, Mass...",39210,9.99 €,28527,83,72881,"2,000,000 .. 5,000,000",517.92 heures,"34,965,000.00 €"
3,Fallout 4 VR,"Fallout 4, the legendary post-apocalyptic adve...",2017-12-12,85.33,PC (Microsoft Windows),"Bethesda Game Studios, Bethesda Softworks",Single player,377160,19.99 €,21120,84,259877,"10,000,000 .. 20,000,000",98.37 heures,"299,850,000.00 €"
4,Albion Online,Albion Online is a sandbox MMORPG set in an op...,2017-07-17,87.50,"Linux, Android, PC (Microsoft Windows), iOS, Mac",Sandbox Interactive,"Multiplayer, Massively Multiplayer Online (MMO)",761890,Gratuit,13121,N/A,1869,"2,000,000 .. 5,000,000",67.07 heures,N/A
5,Star Wars: The Old Republic,Step in to the center of your own Star Wars st...,2011-12-20,85.00,"PC (Microsoft Windows), Mac","Electronic Arts, BioWare Austin, LucasArts, Bi...","Multiplayer, Co-operative, Massively Multiplay...",1286830,Gratuit,5932,85,410,"5,000,000 .. 10,000,000",15.60 heures,N/A
6,Guild Wars 2: Heart of Thorns,The first expansion pack for Guild Wars 2.\nJo...,2015-10-23,85.00,"PC (Microsoft Windows), Mac","ArenaNet, NCSOFT",Massively Multiplayer Online (MMO),1284210,Gratuit,5416,90,338,"2,000,000 .. 5,000,000",37.93 heures,N/A
7,Guild Wars 2: Path of Fire,Guild Wars 2: Path of Fire is the second expan...,2017-09-22,83.75,"PC (Microsoft Windows), Mac",ArenaNet,Massively Multiplayer Online (MMO),1284210,Gratuit,5416,90,338,"2,000,000 .. 5,000,000",37.93 heures,N/A
8,Grim Dawn: Definitive Edition,The Grim Dawn Definitive Edition includes:\n\n...,2019-12-31,85.00,"PC (Microsoft Windows), Xbox One",Crate Entertainment,"Single player, Multiplayer, Co-operative",219990,24.99 €,3202,83,83283,"10,000,000 .. 20,000,000",55.65 heures,"374,850,000.00 €"
9,Dragon Age: Origins,Dragon Age: Origins is a third-person role-pla...,2009-11-03,90.20,"PlayStation 3, PC (Microsoft Windows), Mac, Xb...","Electronic Arts, Edge of Reality, BioWare Edmo...",Single player,1845910,59.99 €,2334,N/A,36556,"2,000,000 .. 5,000,000",50.58 heures,"209,965,000.00 €"


,Nom,Résumé,Date de sortie,Note,Plateformes,Développeurs/Éditeurs,Modes de jeu,AppID,Prix,Joueurs Actuels,Score Metacritic,Recommandations Positives,Propriétaires,Temps de Jeu Moyen,Revenus Approximatifs
0,Baldur's Gate II: Shadows of Amn,Every World has conflict. Good and evil. Frien...,2000-09-21,90.00,"Linux, PC (Microsoft Windows), Mac","BioWare, Black Isle Studios, BioWare Edmonton,...","Single player, Co-operative",1086940,47.99 €,80926,96,655397,"20,000,000 .. 50,000,000",121.70 heures,"1,679,650,000.00 €"
1,Baldur's Gate: Siege of Dragonspear - Collecto...,The Collector's Edition features a limited-edi...,2016-03-31,95.00,"PC (Microsoft Windows), Mac","Overhaul Games, Beamdog",Single player,1086940,47.99 €,80926,96,655380,"20,000,000 .. 50,000,000",121.70 heures,"1,679,650,000.00 €"
2,Final Fantasy XIV Online,Take part in an epic and ever-changing Final F...,2013-08-27,85.33,"Xbox Series X|S, PlayStation 3, PlayStation 4,...","Square Enix Product Development Division 3, Sq...","Single player, Multiplayer, Co-operative, Mass...",39210,9.99 €,28527,83,72881,"2,000,000 .. 5,000,000",517.92 heures,"34,965,000.00 €"
3,Fallout 4 VR,"Fallout 4, the legendary post-apocalyptic adve...",2017-12-12,85.33,PC (Microsoft Windows),"Bethesda Game Studios, Bethesda Softworks",Single player,377160,19.99 €,21120,84,259877,"10,000,000 .. 20,000,000",98.37 heures,"299,850,000.00 €"
4,Albion Online,Albion Online is a sandbox MMORPG set in an op...,2017-07-17,87.50,"Linux, Android, PC (Microsoft Windows), iOS, Mac",Sandbox Interactive,"Multiplayer, Massively Multiplayer Online (MMO)",761890,Gratuit,13121,N/A,1869,"2,000,000 .. 5,000,000",67.07 heures,N/A
